## 資料前處理

In [1]:
import pandas as pd
import numpy as np
import jieba

In [2]:
data = []
lable = []

In [3]:
count = 0
with open("Ch_trainfile_Sentiment_3000.txt", encoding="utf8") as f:
    for sentence in f:
        try:
            sen_list = sentence.split("\t")
            # 如果lable為空值則直接略過，不加入資料集
            lable.append(int(sen_list[0].replace(" ", "")))
            data.append(sen_list[1].replace("\n", ""))
        except ValueError:
            count +=1
print("空值數{}".format(count))

空值數14


In [4]:
len(lable)

2987

In [5]:
lable[1]

1

In [6]:
data[5]

'女装就很少啦'

In [7]:
X = pd.DataFrame(lable,columns=["lable"])

In [8]:
X.insert(1, "text", data)

In [9]:
X.head(10)

,lable,text
0,0,商场几乎都没有活动
1,1,总体感觉蛮好的
2,-1,但是货品实在不咋滴
3,0,人比较少
4,0,还是贵的
5,0,女装就很少啦
6,0,还不知道这家会不会出现呢
7,2,去龙之梦购物还是一站式蛮方便的
8,0,那些BA 也一个个死人脸一样
9,0,但唐宫等位要有耐心哦


In [10]:
# 觀察各類的數量
X["lable"].value_counts() 

 0    1587
 2     586
 1     572
-1     201
-2      38
 3       3
Name: lable, dtype: int64

## 利用jieba斷詞

In [11]:
# stop_word_set = set()
# with open(r'./stopwords.txt', 'r', encoding='utf-8') as sw:
#     for line in sw:
#         stop_word_set.add(line.strip('\n'))

In [12]:
word_cut_list = []
for data_item in data:
    word_list = []
    for word in jieba.cut(data_item):
#         if word not in stop_word_set:
        word_list.append(word)
    content = (' '.join(word_list)).replace("\xa0", "")
    word_cut_list.append(content)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.614 seconds.
Prefix dict has been built succesfully.


In [13]:
word_cut_list

['商场 几乎 都 没有 活动',
 '总体 感觉 蛮 好 的',
 '但是 货品 实在 不 咋 滴',
 '人 比较 少',
 '还是 贵 的',
 '女装 就 很少 啦',
 '还 不 知道 这家 会 不会 出现 呢',
 '去 龙之梦 购物 还是 一站式 蛮 方便 的',
 '那些 BA  也 一个个 死 人脸 一样',
 '但 唐宫 等位 要 有 耐心 哦',
 '整体 环境 不错',
 '主要 是 去 麦吉柯',
 '人 就 没少 过',
 '就是 布局 实在 是 不 喜欢',
 '没有 体力 是 一项 不能 完成 的 任务',
 '综合型 购物 商场',
 '已经 车位 停 了 超满',
 '一条龙 了',
 '其实 去 正大 的 次数 不 多',
 '夏天 的 时候 去过 奥 特莱斯',
 '以前 住 浦东 的 时候 经常 去逛',
 '就 吃 了 点 东西',
 '后面 的 大块 都 是 B 区',
 '人山人海',
 '是 个 休闲 的 好去处',
 '以前 比较 喜欢 在 5 楼 以上 吃饭',
 '商场 很大',
 '一般 去 百货公司',
 '大家 都 有 诸多 变化',
 '购物 也 很 方便',
 '对 不 太 逛得 人 来说 容易 迷路',
 '左右 两边 搞不清楚 ~',
 '在 最 上面 几层 餐厅 非常 多',
 '很大',
 '周六 晚上 去 的',
 '品质 就 不用说 了',
 '失望 哒',
 '格局 还是 不错 的',
 '其他 店 都 没有 什么 顾客',
 '有 免费 停车场',
 '买过 几次 汗衫',
 '看 了 很多',
 '书店',
 '哥哥 说 不错 啊',
 '逛 的 最 多 的 大 商场 就是 这里 了',
 '谁 知道 真的假 的 呢',
 '感觉 有点 小',
 '而且 人暴 多',
 '非常 便捷 的 购物中心',
 '所以 姐妹 们 注意 了',
 '釜山 料理 、 豆捞坊 、 巴 贝拉',
 '有 电影院',
 '样子 也 不怎么样',
 '东西 很 便宜',
 '5 楼 吃 的 东东 挺 多',
 '认真 逛 一天 很难 逛 完 的',
 '综合性 商场',
 '先找 好 回去 的 车站',
 '于是 从那以后 对 正大 实在 有 阴影  也 不敢 去 了',
 '找 不到 想 找 的 地方

## 利用tf-idf及轉換成文字向量 得到特徵值

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [15]:
count_vect=CountVectorizer()
tfidf_transformer=TfidfTransformer()
X_counts = count_vect.fit_transform(word_cut_list)
X_train = tfidf_transformer.fit_transform(X_counts)
y_train = X["lable"]

## k-fold cross validation評估模型效能

In [16]:
# SVM
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
scores = []
kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)
clf = SVC(kernel="linear")
for k, (train, test) in enumerate(kfold):
    clf.fit(X_train[train], y_train[train])
    # 計算5次的分數
    score = clf.score(X_train[test], y_train[test])
    print('Fold: %s,  Acc: %.3f' % (k+1, score))
    scores.append(score)

print('accuracy: %.3f std: %.3f' % (np.mean(scores), np.std(scores)))


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold: 1,  Acc: 0.729
Fold: 2,  Acc: 0.728
Fold: 3,  Acc: 0.717
Fold: 4,  Acc: 0.728
Fold: 5,  Acc: 0.716
accuracy: 0.723 std: 0.006


In [17]:
# Rocchio
from sklearn.neighbors.nearest_centroid import NearestCentroid
scores = []
kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)
clf = NearestCentroid()
for k, (train, test) in enumerate(kfold):
    clf.fit(X_train[train], y_train[train])
    # 計算5次的分數
    score = clf.score(X_train[test], y_train[test])
    print('Fold: %s,  Acc: %.3f' % (k+1, score))
    scores.append(score)

print('accuracy: %.3f std: %.3f' % (np.mean(scores), np.std(scores)))

Fold: 1,  Acc: 0.684
Fold: 2,  Acc: 0.666
Fold: 3,  Acc: 0.643
Fold: 4,  Acc: 0.684
Fold: 5,  Acc: 0.647
accuracy: 0.665 std: 0.017


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [18]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
scores = []
kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)
clf = KNeighborsClassifier(15)
for k, (train, test) in enumerate(kfold):
    clf.fit(X_train[train], y_train[train])
    # 計算5次的分數
    score = clf.score(X_train[test], y_train[test])
    print('Fold: %s,  Acc: %.3f' % (k+1, score))
    scores.append(score)

print('accuracy: %.3f std: %.3f' % (np.mean(scores), np.std(scores)))

Fold: 1,  Acc: 0.577
Fold: 2,  Acc: 0.571
Fold: 3,  Acc: 0.581


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold: 4,  Acc: 0.598
Fold: 5,  Acc: 0.565
accuracy: 0.579 std: 0.011


In [19]:
# decesion tree
from sklearn.tree import DecisionTreeClassifier
scores = []
kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)
clf = DecisionTreeClassifier(random_state=0)
for k, (train, test) in enumerate(kfold):
    clf.fit(X_train[train], y_train[train])
    # 計算5次的分數
    score = clf.score(X_train[test], y_train[test])
    print('Fold: %s,  Acc: %.3f' % (k+1, score))
    scores.append(score)

print('accuracy: %.3f std: %.3f' % (np.mean(scores), np.std(scores)))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold: 1,  Acc: 0.702
Fold: 2,  Acc: 0.684
Fold: 3,  Acc: 0.668
Fold: 4,  Acc: 0.689
Fold: 5,  Acc: 0.677
accuracy: 0.684 std: 0.011


In [20]:
# bayes
from sklearn.naive_bayes import MultinomialNB
scores = []
kfold = StratifiedKFold(n_splits=5).split(X_train, y_train)
clf = MultinomialNB()
for k, (train, test) in enumerate(kfold):
    clf.fit(X_train[train], y_train[train])
    # 計算5次的分數
    score = clf.score(X_train[test], y_train[test])
    print('Fold: %s,  Acc: %.3f' % (k+1, score))
    scores.append(score)

print('accuracy: %.3f std: %.3f' % (np.mean(scores), np.std(scores)))

Fold: 1,  Acc: 0.654
Fold: 2,  Acc: 0.656
Fold: 3,  Acc: 0.653
Fold: 4,  Acc: 0.671
Fold: 5,  Acc: 0.640
accuracy: 0.655 std: 0.010


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
